In [1]:
!pip3 install kaggle
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "content/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)


!mkdir ~/.kaggle
!cp /content/content/kaggle.json ~/.kaggle/kaggle.json

!mkdir ../input
!kaggle competitions download -c retail-case-study-b7 -p ../input

Download 100%.
mkdir: cannot create directory ‘/root/.kaggle’: File exists
mkdir: cannot create directory ‘../input’: File exists
WeatherData.xlsx: Skipping, found more recently modified local copy (use --force to force download)
Train_Kaggle.csv: Skipping, found more recently modified local copy (use --force to force download)
Test_Kaggle.csv: Skipping, found more recently modified local copy (use --force to force download)
Sample_Submission.csv: Skipping, found more recently modified local copy (use --force to force download)
MacroEconomicData.xlsx: Skipping, found more recently modified local copy (use --force to force download)
Events_HolidaysData.xlsx: Skipping, found more recently modified local copy (use --force to force download)
AttributesDescription.xlsx: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline

import math, time, random, datetime

# Data Manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualisation
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier


## Importing required packages
import string
import random
import re
import torch
import torch.nn as nn
from torch.autograd import Variable
import os


# Lets ignore the warnings for now
import warnings
warnings.filterwarnings('ignore')


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input"))

trainPD = pd.read_csv('../input/Train_Kaggle.csv')
testPD  = pd.read_csv('../input/Test_Kaggle.csv')
sampleSubPD  = pd.read_csv('../input/Sample_Submission.csv')

attDescPD = pd.read_excel('../input/AttributesDescription.xlsx')
eventHolidaysPD = pd.read_excel('../input/Events_HolidaysData.xlsx')
macroEconomicsPD = pd.read_excel('../input/MacroEconomicData.xlsx')

['Events_HolidaysData.xlsx', 'Sample_Submission.csv', 'WeatherData.xlsx', 'Train_Kaggle.csv', 'Test_Kaggle.csv', 'MacroEconomicData.xlsx', 'AttributesDescription.xlsx']


In [0]:
xgboostdata = pd.read_csv('/content/xgboostdata.csv')

In [19]:
xgboostdata.head()

,NoOfHolidays,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,Earnings or wages in dollars per hour,Year,Month,Humidity (%) avg,Sales(In ThousandDollars)
0,2.0,14421.752895,14407.053343,233.402,22.05,2009,1,43.0,524.0
1,2.0,14389.200466,14366.176571,234.663,22.22,2009,2,48.0,496.0
2,2.0,14340.701639,14351.786822,235.067,22.22,2009,3,66.0,542.0
3,1.0,14326.815525,14351.601731,235.582,22.13,2009,4,83.0,669.0
4,2.0,14345.904809,14368.123959,235.975,22.04,2009,5,84.0,650.0


In [20]:
xgboostdata.keys()

Index(['NoOfHolidays', 'Monthly Nominal GDP Index (inMillion$)',
       'Monthly Real GDP Index (inMillion$)', 'CPI',
       'Earnings or wages  in dollars per hour', 'Year', 'Month',
       'Humidity (%) avg', 'Sales(In ThousandDollars)'],
      dtype='object')

In [0]:
newKeys = xgboostdata.keys()[:-2]

In [23]:
len(newKeys)

7

In [24]:
# xgboostdata=xgboostdata.drop(['Sales(In ThousandDollars)', 'Humidity (%) avg'],axis=1)
xgboostdata1=xgboostdata[newKeys]
xgboostdata1.keys()

Index(['NoOfHolidays', 'Monthly Nominal GDP Index (inMillion$)',
       'Monthly Real GDP Index (inMillion$)', 'CPI',
       'Earnings or wages  in dollars per hour', 'Year', 'Month'],
      dtype='object')

In [0]:
merged_xgboost_train = pd.merge(xgboostdata1, trainPD, on=['Year', 'Month'])

In [26]:
merged_xgboost_train.head()

,NoOfHolidays,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,Earnings or wages in dollars per hour,Year,Month,ProductCategory,Sales(In ThousandDollars)
0,2.0,14421.752895,14407.053343,233.402,22.05,2009,1,WomenClothing,1755.0
1,2.0,14421.752895,14407.053343,233.402,22.05,2009,1,MenClothing,524.0
2,2.0,14421.752895,14407.053343,233.402,22.05,2009,1,OtherClothing,936.0
3,2.0,14389.200466,14366.176571,234.663,22.22,2009,2,WomenClothing,1729.0
4,2.0,14389.200466,14366.176571,234.663,22.22,2009,2,MenClothing,496.0


In [27]:
merged_xgboost_train.keys()

Index(['NoOfHolidays', 'Monthly Nominal GDP Index (inMillion$)',
       'Monthly Real GDP Index (inMillion$)', 'CPI',
       'Earnings or wages  in dollars per hour', 'Year', 'Month',
       'ProductCategory', 'Sales(In ThousandDollars)'],
      dtype='object')

In [36]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

def processDataForModelling(data,target,drop_cols,is_train=True,split=0.25):
    
    data['ProductCategory'] = LabelEncoder().fit_transform(data['ProductCategory'])
    
    data_1=data.drop(drop_cols,axis=1)
    # One hot Encoding
    data_1=pd.get_dummies(data_1)
    if is_train==True:
        my_imputer = SimpleImputer()
        x = my_imputer.fit_transform(data_1)
        data_1 = pd.DataFrame({
                    'Year':x[:,0],
                    'Month':x[:,1],
                    'NoOfHolidays':x[:,2],
                    'Monthly Nominal GDP Index (inMillion$)':x[:,3],
                    'Monthly Real GDP Index (inMillion$)':x[:,4],
                    'CPI':x[:,5],
                    'Earnings or wages  in dollars per hour':x[:,6],
#                   'Humidity (%) avg':x[:,6],
                    'ProductCategory':x[:,7],
                    'Sales(In ThousandDollars)':x[:,8]
                    })
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1

X_train, X_val, y_train, y_val=processDataForModelling(merged_xgboost_train,'Sales(In ThousandDollars)',drop_cols=[],is_train=True,split=0.2)

Shape of Training Features (172, 8)
Shape of Validation Features  (44, 8)


In [53]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from numpy import loadtxt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X_train, X_val, y_train, y_val


xgb_model = XGBRegressor(objective="reg:linear", random_state=42,max_depth=3, n_estimators = 100)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_val)
mse=mean_squared_error(y_val, y_pred)
print(np.sqrt(mse))

[10:10:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
274.9893957286362


In [60]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split

from numpy import loadtxt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X_train, X_val, y_train, y_val
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
xgb_model = XGBRegressor(random_state=42)
grid = GridSearchCV(xgb_model, params)
grid.fit(X_train, y_train)
y_pred = grid.best_estimator_.predict(X_val)
mse=mean_squared_error(y_val, y_pred)
print(np.sqrt(mse))

[10:14:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [0]:
new_test = pd.read_csv('/content/new_test.csv')

In [61]:
test_data=processDataForModelling(new_test,'Sales(In ThousandDollars)',drop_cols=['Sales(In ThousandDollars)'],is_train=False)
test_data.head()
xgb_test_pred= grid.predict(test_data)
print(xgb_test_pred)

Shape of Test Data (36, 8)
[4251.7876   957.05804 1440.5305  4251.7876   957.05804 1440.5305
 4251.7876   957.05804 1440.5305  4251.7876   957.05804 1440.5305
 4251.7876   957.05804 1440.5305  4251.7876   957.05804 1440.5305
 4251.7876   957.05804 1440.5305  4251.7876   957.05804 1440.5305
 4251.7876   957.05804 1440.5305  4251.7876   957.05804 1440.5305
 4251.7876   957.05804 1440.5305  4251.7876   957.05804 1440.5305 ]


In [62]:
xgb_test_pred = [int(round(each)) for each in xgb_test_pred]
print(xgb_test_pred)

[4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441, 4252, 957, 1441]


In [63]:
# create a submission dataframe and append the relevant columns
submission = pd.DataFrame()
submission['Year'] = [each for each in range(1, 37)]
submission['Sales(In ThousandDollars)'] = xgb_test_pred
submission.to_csv('xgboost_merged_final111.csv', index = False)
submission.head()


,Year,Sales(In ThousandDollars)
0,1,4252
1,2,957
2,3,1441
3,4,4252
4,5,957
